In [1]:
import numpy as np
import pandas as pd
STOP_WRODS=False

In [2]:
#training set and stopwords
train_set = pd.read_csv('labeledmusic.csv')
stop_words = pd.read_csv('stopwords.txt')
stopwords = set(stop_words['stopwords'].values)
lytrain = train_set['text'].values  
Y_train = train_set['mood'].values

In [3]:
def washstr(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [4]:
import nltk
import string
import re

def tokenizer(text):
    lower_txt = washstr(text)
    tokens = lower_txt.split(" ")
    if not STOP_WRODS:
        return tokens
    nonstop = []
    for token in tokens:
        if token not in stopwords:
            nonstop.append(token)
    return nonstop

In [5]:
X_train = []
Y = []

max_len = 0

for i in range(lytrain.shape[0]):
    X_train.append(tokenizer(lytrain[i]))
    max_len = max(max_len, len(X_train[i]))

    if Y_train[i] == "sad":
        Y.append([0,1])
    else:
        Y.append([1,0])

In [6]:
print(len(X_train), max_len)

1199 1266


In [7]:
# fill each lyric, to make it of 1087 tokens
for i in range(len(X_train)):
    while len(X_train[i]) < max_len:
        X_train[i].append(" ")

In [8]:
import gensim
model = gensim.models.Word2Vec(X_train, min_count=1,size=64)

In [9]:
def getwvec(X_train,model):
    x = []
    for i in range(len(X_train)):
        x.append(model.wv[X_train[i]])
    return np.array(x)

In [10]:
x_ = getwvec(X_train,model)
x_ = x_.reshape(-1,x_.shape[1]*x_.shape[2])
print(x_.shape)

(1199, 81024)


In [11]:
y_ = np.array(Y)

In [12]:
print(x_.shape, y_.shape)

(1199, 81024) (1199, 2)


In [13]:
split_frac = 0.8
split_idx = int(len(x_)*0.8)
train_x, val_x = x_[:split_idx], x_[split_idx:]
train_y, val_y = y_[:split_idx], y_[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(959, 81024) 
Validation set: 	(120, 81024) 
Test set: 		(120, 81024)


In [14]:
lstm_size = 32
lstm_layers = 1
batch_size = 110
learning_rate = 0.001

In [15]:
import tensorflow as tf
n_words = len(X_train)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [16]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size =128

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [17]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [18]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,initial_state=initial_state)

In [19]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [20]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [21]:
def get_batches(x, y, batch_size=110):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [23]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%15==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, 'sentiment')

Epoch: 0/10 Iteration: 5 Train loss: 0.214
Epoch: 0/10 Iteration: 10 Train loss: 0.177
Epoch: 1/10 Iteration: 15 Train loss: 0.191
Val acc: 0.703
Epoch: 1/10 Iteration: 20 Train loss: 0.202
Epoch: 2/10 Iteration: 25 Train loss: 0.187
Epoch: 2/10 Iteration: 30 Train loss: 0.233
Val acc: 0.712
Epoch: 3/10 Iteration: 35 Train loss: 0.211
Epoch: 3/10 Iteration: 40 Train loss: 0.202
Epoch: 4/10 Iteration: 45 Train loss: 0.226
Val acc: 0.698
Epoch: 4/10 Iteration: 50 Train loss: 0.165
Epoch: 5/10 Iteration: 55 Train loss: 0.149
Epoch: 5/10 Iteration: 60 Train loss: 0.172
Val acc: 0.725
Epoch: 6/10 Iteration: 65 Train loss: 0.156
Epoch: 6/10 Iteration: 70 Train loss: 0.134
Epoch: 7/10 Iteration: 75 Train loss: 0.101
Val acc: 0.744
Epoch: 7/10 Iteration: 80 Train loss: 0.117
Epoch: 8/10 Iteration: 85 Train loss: 0.126
Epoch: 8/10 Iteration: 90 Train loss: 0.135
Val acc: 0.736
Epoch: 9/10 Iteration: 95 Train loss: 0.119
Epoch: 9/10 Iteration: 100 Train loss: 0.202
Val acc: 0.721


In [24]:
test_acc = []
tf.reset_default_graph()
saver = tf.train.import_meta_graph('sentiment.meta')
with tf.Session(graph=graph) as sess:
    
    saver.restore(sess, "sentiment")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

Test accuracy: 0.736
